# Analysis of COVID-19 Cases in the world


Some requirements:

- Convert data into tidy format
- Export to normalized tables in SQLite3 Database
- Use SQL statements (from within pandas) to retrieve data needed for each visualization

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
# Load raw dataset 
df = pd.read_csv("../data/raw/Data.csv")
df.head()

,Entity,Continent,Latitude,Longitude,Average temperature per year,Hospital beds per 1000 people,Medical doctors per 1000 people,GDP/Capita,Population,Median age,Population aged 65 and over (%),Date,Daily tests,Cases,Deaths
0,Albania,Europe,41.15,20.17,14,2.89,1.29,5353.2,2873457,38,14,2020-02-25,8.0,NaN,NaN
1,Albania,Europe,41.15,20.17,14,2.89,1.29,5353.2,2873457,38,14,2020-02-26,5.0,NaN,NaN
2,Albania,Europe,41.15,20.17,14,2.89,1.29,5353.2,2873457,38,14,2020-02-27,4.0,NaN,NaN
3,Albania,Europe,41.15,20.17,14,2.89,1.29,5353.2,2873457,38,14,2020-02-28,1.0,NaN,NaN
4,Albania,Europe,41.15,20.17,14,2.89,1.29,5353.2,2873457,38,14,2020-02-29,8.0,NaN,NaN


In [4]:
# Categories
df.columns

Index(['Entity', 'Continent', 'Latitude', 'Longitude',
       'Average temperature per year', 'Hospital beds per 1000 people',
       'Medical doctors per 1000 people', 'GDP/Capita', 'Population',
       'Median age', 'Population aged 65 and over (%)', 'Date', 'Daily tests',
       'Cases', 'Deaths'],
      dtype='object')

Since the column name "Entity" is misleading, I will rename it to be country, but first I will check if all the data in that column is actually a country name.

In [5]:
country_list = set(df['Entity'])
set(df['Entity'])

{'Albania',
 'Algeria',
 'Argentina',
 'Armenia',
 'Australia',
 'Austria',
 'Bahrain',
 'Bangladesh',
 'Belarus',
 'Belgium',
 'Bhutan',
 'Bolivia',
 'Bosnia and Herzegovina',
 'Bulgaria',
 'Canada',
 'Cape Verde',
 'Chile',
 'Colombia',
 'Costa Rica',
 'Croatia',
 'Cuba',
 'Cyprus',
 'Denmark',
 'Dominican Republic',
 'Ecuador',
 'El Salvador',
 'Estonia',
 'Ethiopia',
 'Fiji',
 'Finland',
 'France',
 'Ghana',
 'Greece',
 'Guatemala',
 'Hungary',
 'Iceland',
 'India',
 'Indonesia',
 'Iran',
 'Iraq',
 'Ireland',
 'Israel',
 'Italy',
 'Jamaica',
 'Japan',
 'Jordan',
 'Kazakhstan',
 'Kenya',
 'Kuwait',
 'Latvia',
 'Libya',
 'Lithuania',
 'Luxembourg',
 'Madagascar',
 'Malawi',
 'Malaysia',
 'Malta',
 'Mauritania',
 'Mexico',
 'Mongolia',
 'Morocco',
 'Mozambique',
 'Myanmar',
 'Namibia',
 'Nepal',
 'New Zealand',
 'Nigeria',
 'Norway',
 'Oman',
 'Pakistan',
 'Panama',
 'Paraguay',
 'Peru',
 'Philippines',
 'Poland',
 'Portugal',
 'Qatar',
 'Romania',
 'Russia',
 'Rwanda',
 'Saudi Arabia

In [6]:
new_df = df.rename(columns={'Entity': 'Country'})

In [7]:
# Shift (Country, Continent, Date) to the front

shift_keys = ['Country', 'Continent', 'Date']
new_df = new_df[[key for key in shift_keys if key in new_df] + [i for i in new_df if i not in shift_keys]]
new_df

,Country,Continent,Date,Latitude,Longitude,Average temperature per year,Hospital beds per 1000 people,Medical doctors per 1000 people,GDP/Capita,Population,Median age,Population aged 65 and over (%),Daily tests,Cases,Deaths
0,Albania,Europe,2020-02-25,41.15,20.17,14,2.89,1.29,5353.2,2873457,38,14,8.0,NaN,NaN
1,Albania,Europe,2020-02-26,41.15,20.17,14,2.89,1.29,5353.2,2873457,38,14,5.0,NaN,NaN
2,Albania,Europe,2020-02-27,41.15,20.17,14,2.89,1.29,5353.2,2873457,38,14,4.0,NaN,NaN
3,Albania,Europe,2020-02-28,41.15,20.17,14,2.89,1.29,5353.2,2873457,38,14,1.0,NaN,NaN
4,Albania,Europe,2020-02-29,41.15,20.17,14,2.89,1.29,5353.2,2873457,38,14,8.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38467,Zimbabwe,Africa,2021-02-24,-19.02,29.15,20,1.70,0.08,1464.0,16529904,19,3,1804.0,35960.0,1456.0
38468,Zimbabwe,Africa,2021-02-25,-19.02,29.15,20,1.70,0.08,1464.0,16529904,19,3,2965.0,35994.0,1458.0
38469,Zimbabwe,Africa,2021-02-26,-19.02,29.15,20,1.70,0.08,1464.0,16529904,19,3,NaN,36044.0,1463.0
38470,Zimbabwe,Africa,2021-02-27,-19.02,29.15,20,1.70,0.08,1464.0,16529904,19,3,NaN,36058.0,1463.0


In [8]:
# Compare statistics of countries
country_df = new_df.groupby(by='Country').mean(numeric_only = True)
country_df

,Latitude,Longitude,Average temperature per year,Hospital beds per 1000 people,Medical doctors per 1000 people,GDP/Capita,Population,Median age,Population aged 65 and over (%),Daily tests,Cases,Deaths
Country,,,,,,,,,,,,
Albania,41.15,20.17,14.0,2.89,1.29,5353.2,2873457.0,38.0,14.0,1192.831956,2.360359e+04,484.202817
Algeria,28.03,1.66,25.0,1.90,1.83,3974.0,41318142.0,29.0,6.0,1033.333333,4.565502e+04,1547.579096
Argentina,-38.42,-63.62,14.0,5.00,3.91,9912.3,44271041.0,31.0,11.0,15390.744949,7.173078e+05,18499.058659
Armenia,40.07,45.04,11.0,4.20,2.80,4622.7,2930450.0,35.0,11.0,1880.339623,6.676794e+04,1260.817647
Australia,-25.27,133.78,22.0,3.84,3.50,55060.3,24598933.0,37.0,16.0,41312.128319,1.697356e+04,511.164384
...,...,...,...,...,...,...,...,...,...,...,...,...
United States,37.09,-95.71,11.0,2.77,2.57,65297.5,325719178.0,38.0,16.0,912236.261111,7.991328e+06,199648.846995
Uruguay,-32.52,-55.77,16.0,2.80,3.74,16190.1,3456750.0,35.0,15.0,3013.138577,8.799329e+03,108.917160
Vietnam,14.06,108.28,25.0,2.60,0.82,2715.3,95540800.0,32.0,7.0,5753.293103,8.168933e+02,32.737089


In [9]:
# Compare statistics of continents
continent_df = new_df.groupby(by = 'Continent').mean(numeric_only = True)
continent_df

,Latitude,Longitude,Average temperature per year,Hospital beds per 1000 people,Medical doctors per 1000 people,GDP/Capita,Population,Median age,Population aged 65 and over (%),Daily tests,Cases,Deaths
Continent,,,,,,,,,,,,
Africa,3.551885,15.747523,23.348312,1.362825,0.447113,2373.875488,3.465669e+07,21.680748,3.783802,4569.131683,56631.205343,1400.443834
Asia,26.648408,78.826848,20.860524,3.075836,1.657297,14729.942616,1.090739e+08,31.011977,7.025745,58471.975553,351392.865054,7197.241170
Europe,48.987978,15.519167,10.086875,4.928825,3.420736,33769.866981,1.365762e+07,41.531816,18.220805,28850.391774,176893.944399,6272.598683
North America,21.854118,-85.625612,21.450269,2.127629,2.174985,18735.789637,5.445039e+07,32.438021,10.341744,103603.752679,926706.699402,27011.638470
Oceania,-27.652714,161.632464,19.341071,2.951580,2.517446,35401.060268,1.064159e+07,34.163393,12.533036,13486.902864,6648.859066,212.507104
South America,-19.400020,-67.570861,18.278689,2.182247,1.904809,8725.912637,2.305394e+07,30.153005,9.274932,12312.935235,339324.664934,11228.323806


## Normalizing Data

1NF - Remove duplicates  
2NF - Remove partial dependency (i.e. no candidate/super keys)  
3NF - Remove transistive dependency (i.e. no attributes are determined by another attribute other than primary key)

In [10]:
# Remove any duplicates in dataframe (1NF)
new_df = new_df.drop_duplicates()
new_df

,Country,Continent,Date,Latitude,Longitude,Average temperature per year,Hospital beds per 1000 people,Medical doctors per 1000 people,GDP/Capita,Population,Median age,Population aged 65 and over (%),Daily tests,Cases,Deaths
0,Albania,Europe,2020-02-25,41.15,20.17,14,2.89,1.29,5353.2,2873457,38,14,8.0,NaN,NaN
1,Albania,Europe,2020-02-26,41.15,20.17,14,2.89,1.29,5353.2,2873457,38,14,5.0,NaN,NaN
2,Albania,Europe,2020-02-27,41.15,20.17,14,2.89,1.29,5353.2,2873457,38,14,4.0,NaN,NaN
3,Albania,Europe,2020-02-28,41.15,20.17,14,2.89,1.29,5353.2,2873457,38,14,1.0,NaN,NaN
4,Albania,Europe,2020-02-29,41.15,20.17,14,2.89,1.29,5353.2,2873457,38,14,8.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38467,Zimbabwe,Africa,2021-02-24,-19.02,29.15,20,1.70,0.08,1464.0,16529904,19,3,1804.0,35960.0,1456.0
38468,Zimbabwe,Africa,2021-02-25,-19.02,29.15,20,1.70,0.08,1464.0,16529904,19,3,2965.0,35994.0,1458.0
38469,Zimbabwe,Africa,2021-02-26,-19.02,29.15,20,1.70,0.08,1464.0,16529904,19,3,NaN,36044.0,1463.0
38470,Zimbabwe,Africa,2021-02-27,-19.02,29.15,20,1.70,0.08,1464.0,16529904,19,3,NaN,36058.0,1463.0


In [12]:
# 2NF Remove partial dependency (no candidate/super keys)
## Candidate keys are Continent/Latitude/Longtitude; {Latitude + Longtitude} is a composite key as you can determine the country using these two keys

candidate_keys = ['Latitude', 'Longitude', 'Continent']
country_details = new_df[[key for key in candidate_keys]].drop_duplicates().reset_index(drop=True)
country_details

,Latitude,Longitude,Continent
0,41.15,20.17,Europe
1,28.03,1.66,Africa
2,-38.42,-63.62,South America
3,40.07,45.04,Europe
4,-25.27,133.78,Oceania
...,...,...,...
99,37.09,-95.71,North America
100,-32.52,-55.77,South America
101,14.06,108.28,Asia
102,-13.13,27.85,Africa


In [13]:
second_normalized_table = new_df[['Latitude', 'Longitude'] + [i for i in new_df if i not in candidate_keys]]
second_normalized_table

,Latitude,Longitude,Country,Date,Average temperature per year,Hospital beds per 1000 people,Medical doctors per 1000 people,GDP/Capita,Population,Median age,Population aged 65 and over (%),Daily tests,Cases,Deaths
0,41.15,20.17,Albania,2020-02-25,14,2.89,1.29,5353.2,2873457,38,14,8.0,NaN,NaN
1,41.15,20.17,Albania,2020-02-26,14,2.89,1.29,5353.2,2873457,38,14,5.0,NaN,NaN
2,41.15,20.17,Albania,2020-02-27,14,2.89,1.29,5353.2,2873457,38,14,4.0,NaN,NaN
3,41.15,20.17,Albania,2020-02-28,14,2.89,1.29,5353.2,2873457,38,14,1.0,NaN,NaN
4,41.15,20.17,Albania,2020-02-29,14,2.89,1.29,5353.2,2873457,38,14,8.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38467,-19.02,29.15,Zimbabwe,2021-02-24,20,1.70,0.08,1464.0,16529904,19,3,1804.0,35960.0,1456.0
38468,-19.02,29.15,Zimbabwe,2021-02-25,20,1.70,0.08,1464.0,16529904,19,3,2965.0,35994.0,1458.0
38469,-19.02,29.15,Zimbabwe,2021-02-26,20,1.70,0.08,1464.0,16529904,19,3,NaN,36044.0,1463.0
38470,-19.02,29.15,Zimbabwe,2021-02-27,20,1.70,0.08,1464.0,16529904,19,3,NaN,36058.0,1463.0


In [14]:
# 3NF Remove transistive dependency (no attribute is determined by another attribute other than primary key)
## Country -> Latitude, Longtitude, Average temperature per year, Hospital beds per 1000 people, Medical doctors per 1000 people, GDP/Capita, Population, Median age, Population aged 65 and over (%)
final_normalized = second_normalized_table.copy()
final_normalized

,Latitude,Longitude,Country,Date,Average temperature per year,Hospital beds per 1000 people,Medical doctors per 1000 people,GDP/Capita,Population,Median age,Population aged 65 and over (%),Daily tests,Cases,Deaths
0,41.15,20.17,Albania,2020-02-25,14,2.89,1.29,5353.2,2873457,38,14,8.0,NaN,NaN
1,41.15,20.17,Albania,2020-02-26,14,2.89,1.29,5353.2,2873457,38,14,5.0,NaN,NaN
2,41.15,20.17,Albania,2020-02-27,14,2.89,1.29,5353.2,2873457,38,14,4.0,NaN,NaN
3,41.15,20.17,Albania,2020-02-28,14,2.89,1.29,5353.2,2873457,38,14,1.0,NaN,NaN
4,41.15,20.17,Albania,2020-02-29,14,2.89,1.29,5353.2,2873457,38,14,8.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38467,-19.02,29.15,Zimbabwe,2021-02-24,20,1.70,0.08,1464.0,16529904,19,3,1804.0,35960.0,1456.0
38468,-19.02,29.15,Zimbabwe,2021-02-25,20,1.70,0.08,1464.0,16529904,19,3,2965.0,35994.0,1458.0
38469,-19.02,29.15,Zimbabwe,2021-02-26,20,1.70,0.08,1464.0,16529904,19,3,NaN,36044.0,1463.0
38470,-19.02,29.15,Zimbabwe,2021-02-27,20,1.70,0.08,1464.0,16529904,19,3,NaN,36058.0,1463.0


In [15]:
# Save cleaned data into data/clean folder

final_normalized.to_csv("../data/clean/CleanData.csv")